# Скрипты, связанные с смс

In [ ]:
Смс лежат в таблицах:
    прод: qiwi.sms_all
    тест: qiwi.sms_store_partners_q
        
Нет связи отправленных смс и транзакций/счетов, в рамках которых они отправлены.
Можно связывать по следующему условию:
    sms_all.phone = bills.to_agt
И брать смс, отправленные между add_date и commit_date счета.

In [ ]:
Авторизационные смс (т.е. смс, с кодом авторизации, которые отправляются пользователю, который хочет авторизоваться на чекауте
и выбрал способ авторизации смс) можно отобрать по следующим условиям:

message_template='smsconfirm.oauth-otp.sms-text' 
AND
( 
  message_params like 'oplata.qiwi.com%' 
  OR
  message_params like 'Visa QIWI Wallet checkout%'
)

message_params like 'oplata.qiwi.com%' - новый чекаут
message_params like 'Visa QIWI Wallet checkout%' - старый чекаут

In [ ]:
Транзакционные смс (т.е. смс с кодом подтверждения платежа, которые отправляются пользователю
для подтверждения оплаты) можно отобрать так:

message_template='smsconfirm.bill.v2.sms-text' 
OR 
message_template='smsconfirm.bill.v3.sms-text' 
OR
message_template='smsconfirm.bill.v4.sms-text' 

In [ ]:
Смс для восстановления пароля именно на чекауте можно отобрать так:
message_template = 'smsconfirm.restore-password.sms-text'

Для кошелька шаблон другой. Какой - уточнить.

In [ ]:
Смс для регистрации именно можно отобрать так:
message_template = 'smsconfirm.registration.sms-text'

Для чекаута message_params = ****** 
Для кошелька message_params = ****

In [ ]:
Детали:

Смс действительно отправлено, если is_sended = 1

Смс отправлено в Казахстане (это важно при подсчете стоимости смс, т.к. за казахские смс платят казахи)
end_sys in ( 61, 62, 42, 28)

Авторизационные смс до 25 мая 2018 не имели message_params, и поэтому до этой даты невозможно отделить
авторизационные смс чекаута и сайта

In [ ]:
Пример скрипта, который считает число отправленных с чекаута авторизационных смс по дням:

In [ ]:
select count(*), trunc(date_add,'DD')
from qiwi.sms_all
where date_add>=date'2018-05-25'
and message_template='smsconfirm.oauth-otp.sms-text' 
and (message_params like 'oplata.qiwi.com%' or message_params like 'Visa QIWI Wallet checkout%')
and is_sended = 1
and send_sys not in ( 61, 62, 42, 28)
group by trunc(date_add,'DD')

In [ ]:
Пример скрипта, который считает количество авторизационных смс, отправленных при оплате ранее выставленного счета 
с сайта:

In [ ]:
select count(*),trunc(commit_date,'DD')
from qw.bills_q
full outer join qw.sms_store_partners_q
  on bills_q.to_agt = sms_store_partners_q.phone
where commit_date>=date'2018-11-01' 
  and commit_date<date'2018-12-01'
  and extras like '%billref|site%' 
  and extras like '%oauth%'
  and message_template='smsconfirm.oauth-otp.sms-text' 
  and (message_params like 'oplata.qiwi.com%' or message_params like 'Visa QIWI Wallet checkout%')
  and is_sended = 1
  and phone = to_agt
  and (commit_date-date_send)*24*60<60 
  and (commit_date-date_send)*24*60>0
group by trunc(commit_date,'DD')